Before using a detector algorithm, we want to build a baseline.
It will only help to compare the classification performances.

In this notebook, we will build few CNN:
- a CNN from scratch (with 10 dog breeds),
- one Xception model for the 10 dog breeds dataset,
- and a second one Xception model for the whole dataset.

# Importing Keras and connecting to our Google Drive

In [ ]:
import tensorflow.keras as keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data loading, preprocessing and data augmentation

In [ ]:
# Dataset generation (train, validation et test)

# This function will create a train set, a validation set and a test set
# with data augmentation and the preprocessing function of the model,
# which will be used thanks to transfer learning.

path_google_drive = "/content/drive/MyDrive/Gaetan_Travail/ML/Formation_OC_IML/P6/Projet/Data/"

def dataset_for_keras_model(model_preprocessing, batch_size_int, img_size_int, nbr_class):

  # Data augmentation:
  # only for training images
  datagen_train = keras.preprocessing.image.ImageDataGenerator(
      # rescale=1/255,  # normalization
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      validation_split=0,
      preprocessing_function=model_preprocessing
  )

  test_datagen = keras.preprocessing.image.ImageDataGenerator(
      # rescale=1/255,  # normalization
      validation_split=0,
      preprocessing_function=model_preprocessing
  )

  # Train set:
  if (nbr_class == 120):
    path = path_google_drive + "Images_split/train/"
  else:
    path = path_google_drive + "Images_few_" + str(nbr_class) + "/train/"

  x_train = datagen_train.flow_from_directory(
      path,
      target_size=(img_size_int, img_size_int), # resizing
      batch_size=batch_size_int,
      class_mode="categorical",
      shuffle=True,
      seed=0,
      subset='training'
  )

  # Validation set:
  if (nbr_class == 120):
    path = path_google_drive + "Images_split/val/"
  else:
    path = path_google_drive + "Images_few_" + str(nbr_class) + "/val/"  

  x_val = test_datagen.flow_from_directory(
      path,
      batch_size=batch_size_int,
      class_mode="categorical",
      target_size=(img_size_int, img_size_int), # resizing
      seed=0,
      subset='training'
  )

  # Test set:
  if (nbr_class == 120):
    path = path_google_drive + "Images_split/test/"
  else:
    path = path_google_drive + "Images_few_" + str(nbr_class) + "/test/"  

  x_test = test_datagen.flow_from_directory(
      path,
      batch_size=batch_size_int,
      class_mode="categorical",
      target_size=(img_size_int, img_size_int), # resizing
      seed=0,
      subset='training'
  )

  return x_train, x_val, x_test

# CNN from scratch

In [ ]:
nbr_class = 10
IMG_SIZE_ = 150

def build_model_10_dogs():
  input_ = keras.layers.Input(shape=(IMG_SIZE_, IMG_SIZE_, 3))

  #----------------------#

  x = keras.layers.Conv2D(64, 7, padding="same", use_bias=False)(input_)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation("relu")(x)
  x = keras.layers.MaxPool2D(2)(x)

  #----------------------#

  x = keras.layers.Conv2D(256, 3, padding="same", use_bias=False)(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation("relu")(x)
  x = keras.layers.MaxPool2D(2)(x)

  #----------------------#

  x = keras.layers.Conv2D(64, 3, padding="same", use_bias=False)(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation("relu")(x)

  x = keras.layers.Conv2D(256, 3, padding="same", use_bias=False)(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation("relu")(x)

  x = keras.layers.Conv2D(64, 3, padding="same", use_bias=False)(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation("relu")(x)
  x = keras.layers.MaxPool2D(2)(x)

  #----------------------#

  x = keras.layers.Flatten()(x)

  x = keras.layers.Dense(
      units=2048,
      use_bias=False,
      kernel_regularizer=keras.regularizers.l2(0.01)
  )(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation("relu")(x)

  #----------------------#

  x = keras.layers.Dropout(rate=0.4)(x)

  #----------------------#

  x = keras.layers.Dense(
      units=2048,
      use_bias=False,
      kernel_regularizer=keras.regularizers.l2(0.01)
  )(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation("relu")(x)

  #----------------------#

  x = keras.layers.Dropout(rate=0.4)(x)
  
  #----------------------#

  output_ = keras.layers.Dense(
      nbr_class,
      activation="softmax",
      kernel_regularizer=keras.regularizers.l2(0.1)
  )(x)

  #----------------------#

  model = keras.Model(inputs=[input_], outputs=[output_])

  #----------------------#

  model.compile(
        loss="categorical_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        metrics=["accuracy"]
  )

  #----------------------#

  return(model)

In [ ]:
# Dataset for 10 dog breeds
x_train, x_val, x_test = dataset_for_keras_model(
    model_preprocessing=None,
    batch_size_int=10,
    img_size_int=150,
    nbr_class=10
)

# Early stopping
early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True
)

Found 1025 images belonging to 10 classes.
Found 338 images belonging to 10 classes.
Found 350 images belonging to 10 classes.


In [ ]:
model_CNN_from_scratch = build_model_10_dogs()

# Training of the model:
# epochs is very high because we are using an early stopping callback
# -> we don't need to focus on the number of epochs, because early stopping will
# stop the model before it will overfit.

history_CNN_from_scratch = model_CNN_from_scratch.fit(
  x_train,
  epochs=1000,
  batch_size=10,
  validation_data=x_val,
  callbacks=[early_stopping_cb]
)

Epoch 1/1000
103/103 [==============================] - 683s 6s/step - loss: 61.7148 - accuracy: 0.1564 - val_loss: 59.1364 - val_accuracy: 0.2160
Epoch 2/1000
103/103 [==============================] - 11s 106ms/step - loss: 58.3776 - accuracy: 0.1725 - val_loss: 55.8881 - val_accuracy: 0.1686
Epoch 3/1000
103/103 [==============================] - 11s 108ms/step - loss: 55.1651 - accuracy: 0.2019 - val_loss: 52.5092 - val_accuracy: 0.2633
Epoch 4/1000
103/103 [==============================] - 11s 105ms/step - loss: 51.7424 - accuracy: 0.2197 - val_loss: 48.9520 - val_accuracy: 0.2899
Epoch 5/1000
103/103 [==============================] - 11s 106ms/step - loss: 48.2383 - accuracy: 0.2560 - val_loss: 45.6437 - val_accuracy: 0.3018
Epoch 6/1000
103/103 [==============================] - 11s 107ms/step - loss: 44.9072 - accuracy: 0.2454 - val_loss: 43.0757 - val_accuracy: 0.2219
Epoch 7/1000
103/103 [==============================] - 11s 108ms/step - loss: 41.6565 - accuracy: 0.2506 - 

In [ ]:
loss, acc = model_CNN_from_scratch.evaluate(x_test, verbose=0)
print("Test set: ")
print("loss: ", loss)
print("accuracy: ", acc)

Test set: 
loss:  3.601980447769165
accuracy:  0.41428571939468384


Our CNN achieves an accuracy of 41.43 % on the test set.

Now we will use the Xception model thanks to the transfer learning.

# Xception - 10 dog breeds

In [ ]:
# Transfer learning without the top layers
base_model_xception = keras.applications.xception.Xception(
  weights="imagenet",
  include_top=False
)

x = keras.layers.GlobalAveragePooling2D()(base_model_xception.output)

output_ = keras.layers.Dense(10, activation="softmax")(x)

model_xception = keras.Model(inputs=base_model_xception.input, outputs=output_)

for layer in base_model_xception.layers:
  layer.trainable = False

model_xception.compile(
    optimizer = keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# Dataset for 10 dog breeds
x_train_xception, x_val_xception, x_test_xception = dataset_for_keras_model(
    model_preprocessing=keras.applications.xception.preprocess_input,
    batch_size_int=10,
    img_size_int=299,
    nbr_class=10
)

history_xception = model_xception.fit(
    x_train_xception,
    epochs=10,
    batch_size=10,
    validation_data=x_test_xception,
    callbacks=[keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
)

Found 1025 images belonging to 10 classes.
Found 338 images belonging to 10 classes.
Found 350 images belonging to 10 classes.
Epoch 1/10
103/103 [==============================] - 35s 300ms/step - loss: 0.5768 - accuracy: 0.8373 - val_loss: 0.0230 - val_accuracy: 0.9914
Epoch 2/10
103/103 [==============================] - 30s 287ms/step - loss: 0.0369 - accuracy: 0.9868 - val_loss: 0.0148 - val_accuracy: 0.9943
Epoch 3/10
103/103 [==============================] - 30s 292ms/step - loss: 0.0166 - accuracy: 0.9974 - val_loss: 0.0146 - val_accuracy: 1.0000
Epoch 4/10
103/103 [==============================] - 30s 292ms/step - loss: 0.0119 - accuracy: 0.9973 - val_loss: 0.0159 - val_accuracy: 0.9943
Epoch 5/10
103/103 [==============================] - 29s 283ms/step - loss: 0.0064 - accuracy: 0.9993 - val_loss: 0.0142 - val_accuracy: 0.9971
Epoch 6/10
103/103 [==============================] - 29s 282ms/step - loss: 0.0189 - accuracy: 0.9959 - val_loss: 0.0130 - val_accuracy: 0.9943
Epo

In [ ]:
list_model_name.append("xception")

loss, acc = model_xception.evaluate(x_test_xception, verbose=1)

35/35 [==============================] - 3s 83ms/step - loss: 0.0115 - accuracy: 0.9971


The Xception model achieves an accuracy of 99.71 % on the test set.

Xception is pre-trained on ImageNet. Besides, our dog images came from ImageNet too. There is clearly an issue with this dataset and model (it already now the images).
Finally, the small size of the dataset reduces the risk of misclassification.

We will now train the Xception model with the whole dataset, i.e. with 120 dog breeds.

# Xception - 120 dog breeds

In [ ]:
# Transfer learning without the top layers
base_model_xception = keras.applications.xception.Xception(
  weights="imagenet",
  include_top=False
)

x = keras.layers.GlobalAveragePooling2D()(base_model_xception.output)

x = keras.layers.Dropout(0.5)(x)

output_ = keras.layers.Dense(120, activation="softmax")(x)

model_xception_full = keras.Model(inputs=base_model_xception.input, outputs=output_)

for layer in base_model_xception.layers:
  layer.trainable = False

model_xception_full.compile(
    optimizer = keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# Dataset for 102 dog breeds
x_train_xception_full, x_val_xception_full, x_test_xception_full = dataset_for_keras_model(
    model_preprocessing=keras.applications.xception.preprocess_input,
    batch_size_int=10,
    img_size_int=299,
    nbr_class=120
)

history_xception_full = model_xception_full.fit(
    x_train_xception_full,
    epochs=1000,
    batch_size=10,
    validation_data=x_val_xception_full,
    callbacks=[keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)

# Saving this model
model_xception_full.save("/content/drive/MyDrive/Gaetan_Travail/ML/Formation_OC_IML/P6/Projet/transfert_learning_xception_full_19042021.h5")

model_xception_full.evaluate(x_test_xception_full, verbose=1)

Found 12307 images belonging to 120 classes.
Found 4072 images belonging to 120 classes.
Found 4200 images belonging to 120 classes.
Epoch 1/1000
1231/1231 [==============================] - 11295s 9s/step - loss: 1.6762 - accuracy: 0.6517 - val_loss: 0.3241 - val_accuracy: 0.9023
Epoch 2/1000
1231/1231 [==============================] - 296s 241ms/step - loss: 0.3672 - accuracy: 0.8868 - val_loss: 0.3145 - val_accuracy: 0.8991
Epoch 3/1000
1231/1231 [==============================] - 293s 238ms/step - loss: 0.3320 - accuracy: 0.8898 - val_loss: 0.3048 - val_accuracy: 0.9013
Epoch 4/1000
1231/1231 [==============================] - 293s 238ms/step - loss: 0.3031 - accuracy: 0.8975 - val_loss: 0.3138 - val_accuracy: 0.8964
Epoch 5/1000
1231/1231 [==============================] - 292s 238ms/step - loss: 0.2706 - accuracy: 0.9084 - val_loss: 0.3122 - val_accuracy: 0.8996
Epoch 6/1000
1231/1231 [==============================] - 293s 238ms/step - loss: 0.2741 - accuracy: 0.9072 - val_loss

[0.33845940232276917, 0.8930952548980713]

Whit the whole dataset, the Xception model achieves an accuracy of 89.31 % on the test set.